In [1]:
import os

predicted_binding_sites_folder = 'predicted_binding_sites'

prediction_files = list(filter(lambda file: '_predictions' in file , os.listdir(predicted_binding_sites_folder)))

print(len(prediction_files))

49


In [2]:
import pandas as pd
import re

df = pd.read_csv(os.path.join(predicted_binding_sites_folder, prediction_files[0]))

#df.iloc[0,9].strip().split(' ') # list

#[token.split('_')[-1] for token in df.iloc[0,9].strip().split(' ')] #residue ids


In [3]:
binding_residues = set()
for binding_pred_file in prediction_files:
    df = pd.read_csv(os.path.join(predicted_binding_sites_folder, binding_pred_file))
    #print(df.iloc[0,9].strip())
    #print(type(df.iloc[0,9].strip()))
    for token in df.iloc[0,9].strip().split(' '):
        binding_residues.add(token)
binding_residues #binding site residues which are predicted, an unique set

{'A_105',
 'A_106',
 'A_107',
 'A_108',
 'A_117',
 'A_118',
 'A_119',
 'A_120',
 'A_126',
 'A_137',
 'A_173',
 'A_180',
 'A_183',
 'A_184',
 'A_188',
 'A_211',
 'A_212',
 'A_213',
 'A_215',
 'A_216',
 'A_220',
 'A_221',
 'A_223',
 'A_224',
 'A_227',
 'A_230',
 'A_241',
 'A_271',
 'A_299',
 'A_301',
 'A_302',
 'A_303',
 'A_304',
 'A_305',
 'A_306',
 'A_308',
 'A_309',
 'A_310',
 'A_311',
 'A_312',
 'A_313',
 'A_315',
 'A_364',
 'A_369',
 'A_370',
 'A_371',
 'A_372',
 'A_373',
 'A_374',
 'A_375',
 'A_398',
 'A_433',
 'A_434',
 'A_435',
 'A_436',
 'A_437',
 'A_440',
 'A_441',
 'A_442',
 'A_443',
 'A_444',
 'A_447',
 'A_448',
 'A_451',
 'A_481',
 'A_482',
 'A_483',
 'A_484',
 'A_485',
 'A_488',
 'A_50',
 'A_57',
 'A_76',
 'A_78',
 'A_79',
 'A_94'}

In [4]:
s = set()
for binding_pred_file in prediction_files:
    df = pd.read_csv(os.path.join(predicted_binding_sites_folder, binding_pred_file))
    #print(df.iloc[0,9].strip())
    #print(type(df.iloc[0,9].strip()))
    lst = []
    #print(df.iloc[0,9].strip().split(' '))
    
    for token in df.iloc[0,9].strip().split(' '):
        lst.append(token)
        
    s.add(''.join(lst))
    
len(s)

13

In [5]:
len(binding_residues)

76

In [43]:
def compute_centroid(vectors):
    """
        vectors: a list of lists for eg: [[1,2,3], [4,5,6], [7,8,9]]
        note that each inner list is a list of float values
        and a inner list is a row vector in R^3 for eg: given, [1,2,3], 1 is the x-coordinate, 2 is the y-coordinate and 3 is the z-coordinate
        
        returns a  centroid as a row_vector in R^3
    """
    import numpy as np
    #we are going to represent vectors as a 2d matrix, with each row being a row vector in R^3
    two_d_matrix = np.array(vectors)
    n = len(vectors)
    centroid = 1/n * np.sum(two_d_matrix, axis=0)
    return centroid

def get_residue_ids_from_p2rank_predictions(pred_csv):
    """
        pred_csv (str): abs file path to a prediction.csv generated by p2rank
        return a sorted list of residues_id, for eg ['A_1', 'A_2'...]
    """
    import pandas as pd
    
    pred_result_df = pd.read_csv(pred_csv)
    #going to get the top result, a.k.a first row
    
    return sorted(pred_result_df.iloc[0, 9].strip().split(' '), key=lambda token: int(token.split('_')[-1]))

def get_mutant_code_from_pred_csv(pred_csv):
    
    """
        pred_csv (str): for this function, don't need to get abs file path.
        for eg: CYP3A4.3.pdbqt_predictions.csv
        
        return just the mutant code, for eg: CYP3A4.3
    """
    return pred_csv.split('_')[0].replace('.pdbqt', '')

def get_pdb_with_mutant_code(folder, mutant_code):
    """
        folder (str): path to folder which stores pdbs generated by scwrl
        eg: 'pdb_f/scwrl_out'
        mutant_code (str): mutant
    """
    
    mutant_pdbs = os.listdir(folder)
    
    return list(filter(lambda file: mutant_code in file, mutant_pdbs))[0]

def get_vectors_of_residue_r_group(residue_ids, mutant_code, pdb_folder):
    """
        residue_ids (list of str): for eg: ['A_50','A_57','A_76','A_78'], it must be sorted
        mutant_code (str): for eg: CYP3A4.11
        pdb_folder (str): for eg: 'pdb_f/scwrl_out'
        
        return map for eg 'A_50': [vector_of_atom1_res_50, vector_of_atom2_res50] etc

    """
    import os
    #get correct pdb
    pdb_files = os.listdir(pdb_folder)
    mutant_pdb = list(filter(lambda file: mutant_code in file, pdb_files))[0]
    
    import re
    pattern = re.compile('^ATOM')
    back_bone_atoms = set(['N', 'CA', 'O', 'C'])
    
    result = {}
    
    with open(pdb_folder + '/' + mutant_pdb, 'r') as mutant_pdb:
        lines = mutant_pdb.readlines()
        
        for res_id in residue_ids:
            chain, res_num = res_id.split('_')
            result[res_id] = []
            
            for line in lines:
                if pattern.match(line):
                    tokens = re.split("\s{1,}", line)
                    
                    from_pdb_chain_type = tokens[4]
                    from_pdb_res_number = tokens[5]
                    from_pdb_atom = tokens[2]
                    
                    if chain == from_pdb_chain_type and res_num == from_pdb_res_number and not from_pdb_atom in back_bone_atoms:
                        #print(from_pdb_atom)
                        x_coord = float(tokens[6])
                        y_coord = float(tokens[7])
                        z_coord = float(tokens[8])
                        
                        result[res_id].append([x_coord, y_coord, z_coord])
                                    
    return result


def get_vectors_of_all_atoms_of_residues(mutant_code, pdb_folder):
    """
        mutant_code (str): for eg: CYP3A4.11
        pdb_folder (str): for eg: 'pdb_f/scwrl_out'
        
        return map for eg '50': [vector_of_atom1_res_50, vector_of_atom2_res50] etc

    """
    import os
    #get correct pdb
    pdb_files = os.listdir(pdb_folder)
    mutant_pdb = list(filter(lambda file: mutant_code in file, pdb_files))[0]
    
    import re
    pattern = re.compile('^ATOM')
    
    
    result = {}
    
    with open(pdb_folder + '/' + mutant_pdb, 'r') as mutant_pdb:
        lines = mutant_pdb.readlines()
        
            
        for line in lines:
            if pattern.match(line):
                tokens = re.split("\s{1,}", line)
                    
                from_pdb_res_number = tokens[5]
                    
                x_coord = float(tokens[6])
                y_coord = float(tokens[7])
                z_coord = float(tokens[8]) 
                
                if from_pdb_res_number not in result:
                    result[from_pdb_res_number]= [[x_coord, y_coord, z_coord]]
                    continue
                    
                result[from_pdb_res_number].append([x_coord, y_coord, z_coord])
                          
    return result

def get_id_to_resname_map(pdb_file):
    import re
    
    id_to_res_name_map = {}
    
    pattern = re.compile('^ATOM')
    
    with open(pdb_file, 'r') as file:
        lines = file.readlines()
        
        for line in lines:
            
            if pattern.match(line):
                tokens = re.split("\s{1,}", line)
                res_num = tokens[5]
                
                if int(res_num) not in id_to_res_name_map:
                    id_to_res_name_map[int(res_num)] = tokens[3]
            
    return id_to_res_name_map

def get_id_names_map_for_binding_res(res_ids, id_to_res_name_map):
    #res_ids example: A_50', 'A_57', 'A_76', 'A_78', must be sorted
    #id_to_res_name_map: {50: 'HIS', 57: 'HIS'}. i.e residue number : 3 residue letter
    
    id_to_res_name_for_binding_sites = {}
    
    for res_id in res_ids:
        _, res_num = res_id.split('_')
        
        if int(res_num) in id_to_res_name_map:
            id_to_res_name_for_binding_sites[int(res_num)] = id_to_res_name_map[int(res_num)]
    
    return id_to_res_name_for_binding_sites

In [7]:
import numpy as np

a = np.array([[1,2,3], [4,5,6], [7,8,9]])

1/3 * np.sum(a, axis=0)

array([4., 5., 6.])

In [8]:
compute_centroid([[1,2,3], [4,5,6], [7,8,9]])

array([4., 5., 6.])

## Evidence that res id for each mutant may not correspond to the same amino acid residue

In [9]:
hash_map = {} # (res_num, res_name)

for file in os.listdir('pdb_f/scwrl_out'):
    
    hm = get_id_to_resname_map(f'pdb_f/scwrl_out/{file}')
    mt_code = file.replace('.pdb', "")
    res_ids = get_residue_ids_from_p2rank_predictions(f'predicted_binding_sites/{mt_code}.pdbqt_predictions.csv')
    binding_hm = get_id_names_map_for_binding_res(res_ids, hm)
    
    for res_num in binding_hm:
        res_name = hm[res_num]
        
        if (res_num, res_name) not in hash_map:
            hash_map[(res_num, res_name)] = 1
            
        else:
            hash_map[(res_num, res_name)] += 1

sorted(hash_map.keys())

[(50, 'ILE'),
 (57, 'PHE'),
 (76, 'ASP'),
 (78, 'GLN'),
 (79, 'GLN'),
 (94, 'LEU'),
 (105, 'ARG'),
 (106, 'ARG'),
 (107, 'PRO'),
 (107, 'SER'),
 (108, 'PHE'),
 (117, 'ALA'),
 (118, 'ILE'),
 (119, 'ALA'),
 (119, 'CYS'),
 (119, 'LEU'),
 (119, 'PHE'),
 (119, 'SER'),
 (119, 'THR'),
 (119, 'TRP'),
 (119, 'VAL'),
 (120, 'ILE'),
 (120, 'LEU'),
 (120, 'PHE'),
 (120, 'TRP'),
 (126, 'TRP'),
 (137, 'PHE'),
 (173, 'LYS'),
 (180, 'SER'),
 (183, 'VAL'),
 (184, 'ILE'),
 (188, 'SER'),
 (211, 'LEU'),
 (212, 'ALA'),
 (212, 'ARG'),
 (213, 'PHE'),
 (215, 'PHE'),
 (216, 'LEU'),
 (220, 'PHE'),
 (221, 'LEU'),
 (223, 'ILE'),
 (224, 'THR'),
 (227, 'PRO'),
 (230, 'ILE'),
 (241, 'PHE'),
 (271, 'PHE'),
 (299, 'SER'),
 (301, 'ALA'),
 (301, 'ILE'),
 (301, 'PHE'),
 (301, 'TRP'),
 (302, 'PHE'),
 (303, 'ILE'),
 (304, 'ALA'),
 (304, 'PHE'),
 (304, 'TRP'),
 (305, 'ALA'),
 (305, 'GLY'),
 (305, 'PHE'),
 (305, 'SER'),
 (305, 'VAL'),
 (306, 'GLY'),
 (308, 'GLU'),
 (309, 'ALA'),
 (309, 'CYS'),
 (309, 'PHE'),
 (309, 'THR'),
 

## Creating 2d contact matrix for binding residues

In [25]:
for file in os.listdir('predicted_binding_sites'):
    
    if '_predictions' not in file:
        continue
    
    mutant_code = get_mutant_code_from_pred_csv(file)
    res_ids = get_residue_ids_from_p2rank_predictions(f'predicted_binding_sites/{file}')
    residue_vectors_map = get_vectors_of_residue_r_group(res_ids, mutant_code, 'pdb_f/scwrl_out')

    residue_id_centroid_map = {}

    for residue_id in residue_vectors_map:
        list_of_vectors = residue_vectors_map[residue_id]
    
        if len(list_of_vectors) == 0:
            print(residue_id)
            continue
    
        residue_id_centroid_map[residue_id] = compute_centroid(list_of_vectors)

    num_residues = len(residue_id_centroid_map)
    dim = (num_residues, num_residues)
    two_d_matrix_1 = np.zeros(shape=dim)
    print(two_d_matrix_1.shape)

    processed_pairs = set()
    i = 0
    for res_i in residue_id_centroid_map:
        j= 0
    
        for res_j in residue_id_centroid_map:
            if res_j == res_i:
                j += 1
                continue
        
            if (res_i, res_j) in processed_pairs or (res_j, res_i) in processed_pairs:
                j+= 1
                continue
            d_i = residue_id_centroid_map[res_i]
            d_j = residue_id_centroid_map[res_j]
            euclidean_dist = np.linalg.norm(d_i - d_j)
        
            #because 2d map is square and symmetric
            two_d_matrix_1[i, j] = euclidean_dist
            two_d_matrix_1[j, i] = euclidean_dist
        
            processed_pairs.add((res_i, res_j))
            processed_pairs.add((res_j, res_i))
        
            j+= 1
        
        i += 1
    
    np.save(f'2DContactMaps/CYP3A4_Mutants/{mutant_code}', two_d_matrix_1)

A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
A_481
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(68, 68)
A_306
A_436
A_444
(62, 62)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(59, 59)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(60, 60)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(60, 60)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(63, 63)
A_306
A_436
A_444
(62, 62)
A_306
A_436
A_444
(61, 61)
A_305
A_306
A_436
A_444
(60, 60)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(68, 68)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(60, 60)
A_306
A_436
A_444
(61, 61)
A_306
A_436
A_444
(60, 60)
A_306
A_436
A_44

## Creating 2d contact maps for entire protein

In [52]:
import plotly.express as px

for file in os.listdir('pdb_f/scwrl_out'):
        
    mutant_code = file.replace(".pdb", "")
    residue_vectors_map = get_vectors_of_all_atoms_of_residues(mutant_code, 'pdb_f/scwrl_out')
    residue_id_centroid_map = {}

    for residue_id in residue_vectors_map:
        list_of_vectors = residue_vectors_map[residue_id]
        
        if len(list_of_vectors) == 0:
            print(residue_id)
            continue
    
        residue_id_centroid_map[residue_id] = compute_centroid(list_of_vectors)

    num_residues = len(residue_id_centroid_map)
    dim = (num_residues, num_residues)
    two_d_matrix = np.zeros(shape=dim)

    processed_pairs = set()
    i = 0
    for res_i in residue_id_centroid_map:
        j= 0
    
        for res_j in residue_id_centroid_map:
            if res_j == res_i:
                j += 1
                continue
        
            if (res_i, res_j) in processed_pairs or (res_j, res_i) in processed_pairs:
                j+= 1
                continue
            d_i = residue_id_centroid_map[res_i]
            d_j = residue_id_centroid_map[res_j]
            euclidean_dist = np.linalg.norm(d_i - d_j)
        
            #because 2d map is square and symmetric
            two_d_matrix[i, j] = euclidean_dist
            two_d_matrix[j, i] = euclidean_dist
        
            processed_pairs.add((res_i, res_j))
            processed_pairs.add((res_j, res_i))
        
            j+= 1
        
        i += 1

    np.save(f'2DContactMaps/CYP3A4_Mutants_Entire/{mutant_code}', two_d_matrix)

## 2d contact for Entire vs 2d contact for Binding Site Residues

In [9]:
!pip install -U kaleido

Requirement already up-to-date: kaleido in /home/howc/anaconda3/lib/python3.8/site-packages (0.2.1)


In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os 

entire_2d_files = os.listdir('2DContactMaps/CYP3A4_Mutants_Entire')
binding_res_2d_files = os.listdir('2DContactMaps/CYP3A4_Mutants_Binding')

for i in range(49):
    mt_code = entire_2d_files[i].replace('.npy', "")
    
    first_entire = np.load('2DContactMaps/CYP3A4_Mutants_Entire/' + entire_2d_files[i])
    first_binding = np.load('2DContactMaps/CYP3A4_Mutants_Binding/' + binding_res_2d_files[i])
    
    entire_name = f'{mt_code} entire protein 2D map'
    binding_name = f'{mt_code} binding residue protein 2D map'
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=[entire_name, binding_name])

    entire_data = go.Heatmap(
            z=first_entire,
        )

    fig.add_trace(
        entire_data,
        row=1, col=1
    )

    binding_data = go.Heatmap(
            z=first_binding,
    )

    fig.add_trace(
        binding_data,
        row=1, col=2
    )
    fig.update_layout(height=500, width=700)
    fig.write_image(f"2DContactMaps/plots/{mt_code}.jpeg")
    

In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import os 

entire_2d_files = os.listdir('2DContactMaps/CYP3A4_Mutants_Entire')
binding_res_2d_files = os.listdir('2DContactMaps/CYP3A4_Mutants_Binding')

fig = make_subplots(rows=1, cols=2, subplot_titles=['Entire protein vs Binding residues 2D contact maps'])

for i in range(49):
    mt_code = entire_2d_files[i].replace('.npy', "")
    
    first_entire = np.load('2DContactMaps/CYP3A4_Mutants_Entire/' + entire_2d_files[i])
    first_binding = np.load('2DContactMaps/CYP3A4_Mutants_Binding/' + binding_res_2d_files[i])
    
    entire_name = f'{mt_code} entire'
    binding_name = f'{mt_code} binding'
    
    entire_data = go.Heatmap(
            z=first_entire,
            name=entire_name,
            visible = 'legendonly'
        )

    fig.add_trace(
        entire_data,
        row=1, col=1
    )

    binding_data = go.Heatmap(
            z=first_binding,
            name=binding_name,
            visible = 'legendonly'
    )

    fig.add_trace(
        binding_data,
        row=1, col=2
    )

    
steps = []

prev = 0

for i in range(0, 49, 2):
    
    step = dict(
        method = 'restyle',  
        args = ['visible', ['legendonly'] * len(fig.data)],
    )

    step['args'][1][i] = True
    step['args'][1][i + 1] = True
    steps.append(step)

sliders = [dict(
    steps = steps,
)]

fig.layout.sliders = sliders

fig.show()

In [10]:
len(steps)
binding_res_2d_files

['CYP3A4-S119F.npy',
 'CYP3A4-A370F.npy',
 'CYP3A4-T433S.npy',
 'CYP3A4-F304W.npy',
 'CYP3A4-F304A.npy',
 'CYP3A4-M371I.npy',
 'CYP3A4.3.npy',
 'CYP3A4-I120F.npy',
 'CYP3A4-I120L.npy',
 'CYP3A4-I369V.npy',
 'CYP3A4-T309F.npy',
 'CYP3A4-A305G.npy',
 'CYP3A4-T363M.npy',
 'CYP3A4-C98A.npy',
 'CYP3A4-L373H.npy',
 'CYP3A4-T309A.npy',
 'CYP3A4-A370V.npy',
 'CYP3A4-T309V.npy',
 'CYP3A4-N104D.npy',
 'CYP3A4-I120W.npy',
 'CYP3A4-I301A.npy',
 'CYP3A4-S119C.npy',
 'CYP3A4-P107S.npy',
 'CYP3A4-L373A.npy',
 'CYP3A4-A305S.npy',
 'CYP3A4-I301W.npy',
 'CYP3A4-S119T.npy',
 'CYP3A4-I301F.npy',
 'CYP3A4-A305V.npy',
 'CYP3A4-T103A.npy',
 'CYP3A4-L373F.npy',
 'CYP3A4-A370C.npy',
 'CYP3A4-C98S.npy',
 'CYP3A4.11.npy',
 'CYP3A4.12.npy',
 'CYP3A4-V376T.npy',
 'CYP3A4-S119V.npy',
 'CYP3A4-I369F.npy',
 'CYP3A4-A305F.npy',
 'CYP3A4-S119W.npy',
 'CYP3A4-S119A.npy',
 'CYP3A4-S312C.npy',
 'CYP3A4-T309C.npy',
 'CYP3A4-S119L.npy',
 'CYP3A4-R212A.npy',
 'CYP3A4-C98F.npy',
 'CYP3A4-L373V.npy',
 'CYP3A4-E374Q.npy',
 'CYP

In [ ]:
import plotly.graph_objs as go
from plotly.tools import make_subplots

fig = make_subplots(1, 2)

fig.add_scatter(y=[1, 3, 2], row=1, col=1, visible=True)
fig.add_scatter(y=[3, 1, 1.5], row=1, col=1, visible='legendonly')
fig.add_scatter(y=[2, 2, 1], row=1, col=1, visible='legendonly')
fig.add_scatter(y=[1, 3, 2], row=1, col=2, visible=True)
fig.add_scatter(y=[1.5, 2, 2.5], row=1, col=2, visible='legendonly')
fig.add_scatter(y=[2.5, 1.2, 2.9], row=1, col=2, visible='legendonly')

print(len(fig.data))

steps = []
for i in range(3):
    step = dict(
        method = 'restyle',  
        args = ['visible', ['legendonly'] * len(fig.data)],
    )
    #step['args'][1][i] = True
    #step['args'][1][i+3] = True
    steps.append(step)

sliders = [dict(
    steps = steps,
)]

fig.layout.sliders = sliders
fig.show()

In [21]:
res_ids = get_residue_ids_from_p2rank_predictions('predicted_binding_sites/CYP3A4-S119T.pdbqt_predictions.csv')
residue_ids = []

for res_id in res_ids:
    residue_ids.append(res_id.split('_')[-1])

s1 = '+'.join(residue_ids)
s1

'50+57+76+78+79+94+105+106+107+108+118+119+120+126+137+180+183+184+188+212+213+215+216+220+221+223+224+227+230+241+271+299+301+302+303+304+305+306+309+310+312+313+364+369+370+371+372+373+374+375+434+435+436+437+440+441+442+443+444+447+448+451+482+483'